# Modelling the data

In [2]:
# Load the data

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
import collections
import seaborn as sns

df = pd.read_csv('../../data/df_main.csv')

ModuleNotFoundError: No module named 'pandas'

In [ ]:
df.loc[(df['year'] == 2004) & (df['from_code2'] == 'GB')].shape

## XGBoost to predict if score given

First step in the hurdle model.

Example: https://www.kaggle.com/code/sriharinitumu/light-gbm-with-hurdle-modelling


In [ ]:
df['received_vote'] = df['points'].apply(lambda x: 1 if x > 0 else 0)
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 4))


# Bar chart for vote occurrences
sns.countplot(x="points", data=df, ax=ax1)
ax1.set_ylabel('Count', fontsize=12)
ax1.set_xlabel('Score', fontsize=12)
ax1.set_title('Number of votes', fontsize=12)

sns.countplot(x="received_vote", data=df, ax=ax2)
ax2.set_ylabel('Count', fontsize=12)
ax2.set_xlabel('Received vote', fontsize=12)
ax2.set_title('Number of votes', fontsize=12)

plt.show()

As you can see we have a lot of zeroes....this probably makes this problem appropriate for a hurdle model. Where we first decide if a score is given, and then if it is, we predict the score.

In [ ]:
df.columns

In [ ]:
# scale some of the features

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler


df_xgboost = df[['year', 'from_code2', 'points', 'to_code2',
        'Contains_English',
       'Contains_NonEnglish', 'Contains_Multiple_Languages',
       'Number_of_Languages', 'Contains_Own_Language', 'Contains_Voting_Language', 'gender',
       'prop_emigrants_v2p',  'prop_emigrants_p2v', 'has_border',
       'comps_without_win', 'received_vote']]

df_xgboost['has_border'] = df_xgboost['has_border'].fillna(0)

# log10 scale the prop_emigrants column
df_xgboost['prop_emigrants_v2p'] = df_xgboost['prop_emigrants_v2p'].apply(lambda x: np.log10(x+4e-8))
df_xgboost['prop_emigrants_p2v'] = df_xgboost['prop_emigrants_p2v'].apply(lambda x: np.log10(x+4e-8))


# apply the standard scaler to prop_emigrants and comps_without_win
scaler = StandardScaler()
df_xgboost[['prop_emigrants_v2p']] = scaler.fit_transform(df_xgboost[['prop_emigrants_v2p']])
df_xgboost[['prop_emigrants_p2v']] = scaler.fit_transform(df_xgboost[['prop_emigrants_p2v']])

scaler = MinMaxScaler()
df_xgboost[['comps_without_win']] = scaler.fit_transform(df_xgboost[['comps_without_win']])


fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(10, 4))

# histogram of the log scaled prop_emigrants column
df_xgboost['comps_without_win'].hist(bins=20, ax=ax1)
df_xgboost['prop_emigrants_v2p'].hist(bins=200, ax=ax2)
df_xgboost['prop_emigrants_p2v'].hist(bins=200, ax=ax3)

In [ ]:

from numpy import loadtxt
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

def basic_xgboost(df_input, seed, test_size, predictor_column='received_vote'):
    X = df_input[['from_code2', 'to_code2', 'points',
        'Contains_English',
       'Contains_NonEnglish', 'Contains_Multiple_Languages',
       'Number_of_Languages', 'Contains_Own_Language', 'Contains_Voting_Language', 'gender',
       'prop_emigrants_v2p',  'prop_emigrants_p2v','has_border',
       'comps_without_win', "received_vote"]]



    Y = df_input[predictor_column]


    # one hot encode the gender, from country and code
    for j in ['gender', 'from_code2', 'to_code2']:
        j_text = '_voting' if j == 'from_code2' else ''
        for i in X[j].unique():
            output_binary = X[j].apply(lambda x: 1 if x == i else 0)
            X[i+j_text] = output_binary

        X = X[X.columns.drop(j)]

    # convert Y to int
    Y = Y.astype(int)

    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=test_size, random_state=seed, stratify=X['points'])

    test_all = X_test

    # remove 'Votes' and 'received_vote' columns from X_train
    X_train = X_train[X_train.columns.drop('points')]
    X_train = X_train[X_train.columns.drop('received_vote')]
    X_test = X_test[X_test.columns.drop('points')]
    X_test = X_test[X_test.columns.drop('received_vote')]

    model = xgb.XGBClassifier()
    model.fit(X_train, y_train)

    return model, X_test, y_test, test_all

def model_predictions(model, X_test, y_test):
    Xt = X_test.copy()
    y_pred = model.predict(Xt)
    predictions = [round(value) for value in y_pred]

    Xt['prob'] = model.predict_proba(Xt)[:,1]
    Xt['predictions'] = predictions
    Xt['actual'] = y_test
    return Xt

def model_evalutation(df_pred):
    # evaluate predictions
    accuracy = accuracy_score(df_pred['actual'], df_pred['predictions'])
    print("Accuracy: %.2f%%" % (accuracy * 100.0))

    # print a confusion matrix heatmap

    cm = confusion_matrix(df_pred['actual'], df_pred['predictions'])
    sns.heatmap(cm, annot=True, fmt="d")

    # add title and axis labels
    plt.title('Confusion matrix' + "(Accuracy={:.3f})".format(accuracy))
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
model_basic, X_test, y_test, test_all = basic_xgboost(df_xgboost, seed=7, test_size=0.33)
predictions = model_predictions(model_basic, X_test, y_test)
model_evalutation(predictions)

In [ ]:
predictions.head()

In [ ]:
for i in  test_all.columns:
    print(i)

In [ ]:
import matplotlib
import xgboost as xgb

graph = xgb.to_graphviz(model_basic, num_trees=1, rankdir='LR')

fig, ax = plt.subplots(figsize=(30, 50))
xgb.plot_tree(model_basic, num_trees=1, ax=ax)

## Ranking Scores

So of those, lets subset to only those with a score

In [ ]:
test_all.head()

In [ ]:
import xgboost as xgb
from scipy.stats import rankdata
from sklearn.metrics import confusion_matrix
import seaborn as sns
from sklearn.model_selection import GroupShuffleSplit
from scipy.stats import spearmanr
import numpy as np


def xgboost_rank_model(df_input, test_size=0.33, seed=7):

    df2 = df_input[['year', 'from_code2', 'to_code2', 'points',
        'Contains_English',
       'Contains_NonEnglish', 'Contains_Multiple_Languages',
       'Number_of_Languages', 'Contains_Own_Language', 'Contains_Voting_Language', 'gender',
       'prop_emigrants_v2p',  'prop_emigrants_p2v','has_border',
       'comps_without_win']]

    #df2 = df2.loc[df2['Votes'] > 0]

    #df2['has_border'] = df2['has_border'].fillna(0)
    df2 = df2.rename(columns={"points": 'rank'})
    #df2 = df2.rename(columns={"From country": 'id'})

    df2['rank'] = df2['rank'].astype(int) - 1

    df2['id'] = df2['from_code2'].astype(str) + df2['year'].astype(str)

    # one hot encode the gender, from country and code
    for j in ['gender', 'to_code2', 'from_code2']:
        j_text = "_voting" if j == 'from_code2' else ''
        for i in df2[j].unique():
            df2[i+j_text] = df2[j].apply(lambda x: 1 if x == i else 0)

        df2 = df2[df2.columns.drop(j)]

    #df2= df2[df2.columns.drop('From country')]
    df2= df2[df2.columns.drop('year')]

    gss = GroupShuffleSplit(test_size=test_size, n_splits=1, random_state = seed).split(df2, groups=df2['id'])

    X_train_inds, X_test_inds = next(gss)

    train_data= df2.iloc[X_train_inds]
    X_train = train_data.loc[:, ~train_data.columns.isin(['id','rank'])]
    y_train = train_data.loc[:, train_data.columns.isin(['rank'])]

    groups = train_data.groupby('id').size().to_frame('size')['size'].to_numpy()

    test_data= df2.iloc[X_test_inds]
    test_data = test_data[X_train.columns.tolist() + ['rank', 'id']]

    #We need to keep the id for later predictions
    # X_test = test_data.loc[:, ~test_data.columns.isin(['rank'])]
    # y_test = test_data.loc[:, test_data.columns.isin(['rank'])]

    model = xgb.XGBRanker(  
        tree_method='hist',
        booster='gbtree',
        objective='rank:pairwise',
        random_state=7, 
        learning_rate=0.1,
        colsample_bytree=0.9, 
        eta=0.05, 
        max_depth=6, 
        n_estimators=110, 
        subsample=0.75 
    )

    model.fit(X_train, y_train, group=groups, verbose=True)

    return model, test_data, train_data

 

def ranked_model_predictions(model, test_set):
    test = test_set.copy()
    
    test['prediction_rel'] = np.nan
    test['predictions'] = np.nan

    for i in test['id'].unique():
        sub_test = test.loc[test['id'] == i, ~test.columns.isin(['id', 'rank', 'prediction_rel', 'predictions'])]

        preds = model.predict(sub_test)

        # get the order of the predictions 
        res = rankdata(preds, method='ordinal')
        test.loc[test['id'] == i, 'prediction_rel'] = res

        # create a score for top 10 predictions getting 1:10 and others 0
        top10 = test.loc[test['id'] == i, 'prediction_rel'].nlargest(10).values


        test.loc[test['id'] == i, 'predictions'] = test.loc[test['id'] == i, 'prediction_rel'].apply(lambda x: x if x in top10 else 0)
        
        # if not zero subtract min(top10) 
        test.loc[test['id'] == i, 'predictions'] = test.loc[test['id'] == i, 'predictions'].apply(lambda x: x - max(top10) + 10 if x != 0 else 0)

        # if 10 set to 12
        test.loc[test['id'] == i, 'predictions'] = test.loc[test['id'] == i, 'predictions'].apply(lambda x: 12 if x == 10 else x)
        test.loc[test['id'] == i, 'predictions'] = test.loc[test['id'] == i, 'predictions'].apply(lambda x: 10 if x == 9 else x)

    test['actual'] = test['rank'] + 1


    accuracy = accuracy_score(test['actual'], test['predictions'])
    print("Accuracy: %.2f%%" % (accuracy * 100.0))

    return test

In [ ]:
model_ranked, test_data, train_data = xgboost_rank_model(df_xgboost.loc[df_xgboost['points'] > 0], seed=7, test_size=0.33)
out = ranked_model_predictions(model_ranked, test_data)

In [ ]:
model_ranked_all, test_data_all, train_data_all = xgboost_rank_model(df_xgboost, seed=7, test_size=0.33)
out_all = ranked_model_predictions(model_ranked_all, test_data_all)

In [ ]:
out_all['predictions'].value_counts()

### Plots

In [ ]:
def violins(pred):
    prediction_df = pred.copy()
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 7)) 

    if(max(prediction_df['actual']) < 12):    
        if(max(prediction_df['actual']) == 11):
            prediction_df['predictions'] = prediction_df['predictions'] + 1
            prediction_df['actual'] = prediction_df['actual'] + 1
        prediction_df['predictions'] = prediction_df['predictions'].apply(lambda x: 12 if x == 10 else x)
        prediction_df['predictions'] = prediction_df['predictions'].apply(lambda x: 10 if x == 9 else x)
        prediction_df['actual'] = prediction_df['actual'].apply(lambda x: 12 if x == 10 else x)
        prediction_df['actual'] = prediction_df['actual'].apply(lambda x: 10 if x == 9 else x)

    # violin plot of predictions for each rank
    sns.violinplot(x="actual", y="predictions", data=prediction_df, order=range(0, 13), ax=ax1)
    sns.violinplot(y="actual", x="predictions", data=prediction_df, order=range(0, 13), ax=ax2)

    # add a best fit line - scale is wrong because numeric on top of categorical
    sns.regplot(x="actual", y="predictions", data=prediction_df, scatter=False, color='black', ax=ax1)
    sns.regplot(y="actual", x="predictions", data=prediction_df, scatter=False, color='black', ax=ax2)


    # spearman correlation
    corr, _ = spearmanr(prediction_df['actual'], prediction_df['predictions'])

    fig.suptitle('Predicted Score vs Actual Score (r = ' + str(round(corr, 4)) + ')', fontsize=15)

    # set x label
    ax1.set_xlabel('Actual Score')
    ax1.set_ylabel('Predicted Score')
    ax2.set_ylabel('Actual Score')
    ax2.set_xlabel('Predicted Score')

def cm_heatmap (prediction_df, title='Confusion matrix', ax=None):
    cm = confusion_matrix(prediction_df['actual'], prediction_df['predictions'])

    plot = sns.heatmap(cm, annot=True, fmt='g', ax=ax, vmin=0, vmax=100)

    if(len(cm[[0]][0]) == 11) : 
    # change the tick labels
        plot.set_xticks(np.arange(0, 11, 1)+0.5, np.arange(0, 9, 1).tolist() + [10, 12])
        plot.set_yticks(np.arange(0, 11, 1)+0.5, np.arange(0, 9, 1).tolist() + [10, 12])
    else: 
        plot.set_xticks(np.arange(0, 10, 1)+0.5, np.arange(1, 9, 1).tolist() + [10, 12])
        plot.set_yticks(np.arange(0, 10, 1)+0.5, np.arange(1, 9, 1).tolist() + [10, 12])

    plot.set_title(title)
    plot.set_ylabel('Actual Score')
    plot.set_xlabel('Predicted Score')

In [ ]:
violins(out)

In [ ]:
violins(out_all.loc[out_all['actual'] > 0])

In [ ]:
cm_heatmap(out)

In [ ]:
cm_heatmap(out_all)

See [here](https://discuss.xgboost.ai/t/evaluating-xgboost-ranking/959/2) for description of prediction scores

The 0 predictions are coming up when we have more that 10 non-zeroes (due to draws). So this actually works quite well I think.

### Quickly see how this compares to a non pairwise rank model


In [ ]:
df_votes = df_xgboost.loc[df_xgboost['points'] > 0]

df_votes['Votes2'] = df_votes["points"] - 1
df_votes['Votes2'] = df_votes['Votes2'].apply(lambda x: 8 if x == 9 else x)
df_votes['Votes2'] = df_votes['Votes2'].apply(lambda x: 9 if x == 11 else x)

model, X_test, y_test, t2 = basic_xgboost(df_votes, seed=7, test_size=0.33, predictor_column='Votes2')
predictions = model_predictions(model, X_test, y_test)
model_evalutation(predictions)

In [ ]:
violins(predictions)

In [ ]:
# ranked and unranked plot side by side
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 7))
fig.suptitle('Confusion Matrix for Ranked and Unranked Predictions')

# ranked plot
cm_heatmap(out, title='Ranked Predictions', ax=ax1)
cm_heatmap(predictions, title='Unranked Predictions', ax=ax2)

# Combine the hurdles

In [ ]:
df_xgboost.head()

In [ ]:
df_xgboost['id'] = df_xgboost['from_code2'] + df_xgboost['year'].astype(str)

df_xgboost['rank'] = df_xgboost['points']

df_xgboost['rank'] = df_xgboost['rank'].apply(lambda x: 9 if x == 10 else x)
df_xgboost['rank'] = df_xgboost['rank'].apply(lambda x: 10 if x == 12 else x)

#df_xgboost['rank'].value_counts()
df_xgboost['rank'] = df_xgboost['rank'].astype(int)

In [ ]:
df_hurdle = df_xgboost.copy()

#df_hurdle['has_border'] = df_hurdle['has_border'].fillna(0)

df_hurdle.head()


In [ ]:
# one hot encode the gender, from country and code
for j in ['gender', 'to_code2', 'from_code2']:
    j_text = "_voting" if j == 'from_code2' else ''
    for i in df_hurdle[j].unique():
        df_hurdle[i+j_text] = df_hurdle[j].apply(lambda x: 1 if x == i else 0)

    df_hurdle = df_hurdle[df_hurdle.columns.drop(j)]

#df2= df2[df2.columns.drop('From country')]
df_hurdle= df_hurdle[df_hurdle.columns.drop('year')]

df_hurdle.head()

In [ ]:
from sklearn.model_selection import GroupShuffleSplit
from sklearn.model_selection import train_test_split

# select a random sample of 33% of the unique values of df_hurdle['id']
test_groups = np.random.choice(df_hurdle['id'].unique(), size=int(len(df_hurdle['id'].unique())*0.33), replace=False)
train_groups = np.setdiff1d(df_hurdle['id'].unique(), test_groups)

train_data = df_hurdle.loc[df_hurdle['id'].isin(train_groups)]
test_data = df_hurdle.loc[df_hurdle['id'].isin(test_groups)]

X_train = train_data.loc[:, ~train_data.columns.isin(['id','rank', 'points', 'received_vote'])]
y_train = train_data.loc[:, train_data.columns.isin(['received_vote'])]

X_test = test_data.loc[:, ~test_data.columns.isin(['id','rank', 'points', 'received_vote'])]
y_test = test_data.loc[:, test_data.columns.isin(['received_vote'])]


In [ ]:
model = xgb.XGBClassifier()
hurdle1_model = model.fit(X_train, y_train)

In [ ]:
hurdle1_df = model_predictions(hurdle1_model, X_test,  y_test)

# merge test_data and hurdle1_df
cols = [i for i in test_data.columns if i not in hurdle1_df.columns]
hurdle1_df = pd.merge(hurdle1_df, test_data[cols], left_index=True, right_index=True)

hurdle1_df.head()

In [ ]:
model_evalutation(hurdle1_df)

In [ ]:
df2 = hurdle1_df.loc[hurdle1_df['predictions'] > 0]

# remove predictions, actual and received_vote columns
df2 = df2[df2.columns.drop(['predictions', 'actual', "points", 'received_vote'])]

print(df2['rank'].value_counts())

#df2 = df2.rename(columns={"Votes": 'rank'})
#df2 = df2.rename(columns={"From country": 'id'})

#df2['rank'] = df2['rank'].astype(int) - 1


# select a random sample of 33% of the unique values of df_hurdle['id']
test_groups = np.random.choice(df2['id'].unique(), size=int(len(df2['id'].unique())*0.33), replace=False)
train_groups = np.setdiff1d(df2['id'].unique(), test_groups)

train_data = df2.loc[df2['id'].isin(train_groups)]
test_data = df2.loc[df2['id'].isin(test_groups)]

X_train = train_data.loc[:, ~train_data.columns.isin(['id','rank', 'points', 'received_vote'])]
y_train = train_data.loc[:, train_data.columns.isin(['rank'])]

X_test = test_data.loc[:, ~test_data.columns.isin(['id','rank', 'points', 'received_vote'])]
y_test = test_data.loc[:, test_data.columns.isin(['rank'])]

print(test_data['id'].value_counts())
print(train_data['id'].value_counts())

groups = train_data.groupby('id').size().to_frame('size')['size'].to_numpy()

model_ranked = xgb.XGBRanker(  
    tree_method='hist',
    booster='gbtree',
    objective='rank:pairwise',
    random_state=7, 
    learning_rate=0.1,
    colsample_bytree=0.9, 
    eta=0.05, 
    max_depth=6, 
    n_estimators=110, 
    subsample=0.75 
)

model_ranked.fit(X_train, y_train, group=groups, verbose=True)

In [ ]:
from scipy.stats import rankdata
from sklearn.metrics import accuracy_score

test = test_data.copy()

test['prediction_rel'] = np.nan
test['predictions'] = np.nan

for i in test['id'].unique():
    sub_test = test.loc[test['id'] == i, ~test.columns.isin(['id', 'rank', 'prediction_rel', 'predictions'])]

    preds = model.predict(sub_test)

    # get the order of the predictions 
    res = rankdata(preds, method='ordinal')
    test.loc[test['id'] == i, 'prediction_rel'] = res

    # create a score for top 10 predictions getting 1:10 and others 0
    top10 = test.loc[test['id'] == i, 'prediction_rel'].nlargest(10).values


    test.loc[test['id'] == i, 'predictions'] = test.loc[test['id'] == i, 'prediction_rel'].apply(lambda x: x if x in top10 else 0)
    
    # if not zero subtract min(top10) 
    test.loc[test['id'] == i, 'predictions'] = test.loc[test['id'] == i, 'predictions'].apply(lambda x: x - max(top10) + 10 if x != 0 else 0)

    # if 10 set to 12
    #test.loc[test['id'] == i, 'predictions'] = test.loc[test['id'] == i, 'predictions'].apply(lambda x: 12 if x == 10 else x)
    #test.loc[test['id'] == i, 'predictions'] = test.loc[test['id'] == i, 'predictions'].apply(lambda x: 10 if x == 9 else x)

test['actual'] = test['rank'] 


accuracy = accuracy_score(test['actual'], test['predictions'])
print("Accuracy: %.2f%%" % (accuracy * 100.0))

predictions2 = test

In [ ]:
predictions2['actual'].value_counts()

In [ ]:
for i in hurdle1_df.columns:
    print(i)

In [ ]:
p1 = hurdle1_df.copy()


cols = [col for col in p1.columns if '_voting' in col]
cols2 = [col for col in p1.columns if '_voting' not in col]


# wide format _voting columns to long format
hurdle_input2 = pd.melt(p1, id_vars=cols2, 
                       value_vars= cols, var_name='from_code2')

hurdle_input2 = hurdle_input2.loc[hurdle_input2['value'] ==1]



p1['from_code2'] = hurdle_input2['from_code2'].str.replace('_voting', '').tolist()
print(p1['from_code2'])
p1 = p1.drop(cols, axis=1)


cols2 = [#'year', 
       'from_code2','received_vote', #'to_code2', 
       'points', 'rank',
       'Contains_English', 'id',
       'has_border',
       'Contains_NonEnglish', 'Contains_Multiple_Languages',
       'Number_of_Languages', 'Contains_Own_Language', 'Contains_Voting_Language', 
       'prop_emigrants_v2p',  'prop_emigrants_p2v', 
       'comps_without_win', 'group', 'female', 'male', 
       'predictions', 'actual']

cols = [col for col in p1.columns if col not in cols2]

# print p1.columns not in cols2
print([col for col in p1.columns if col not in cols2])

hurdle_input4 = pd.melt(p1, id_vars=cols2, 
                       value_vars= cols, var_name='to_code2')

hurdle_input4 = hurdle_input4.loc[hurdle_input4['value'] == 1]

# add hurdle_input2['From country'] to hurdle_input

p1['to_code2'] = hurdle_input4['to_code2'].tolist()

predictions1 = p1


In [ ]:
pred1 = predictions1.loc[predictions1['predictions'] == 0]

#print(pred1['actual'].value_counts())

pred1 = pred1[['predictions', 'rank', 'id']] # , 'code', 'From country'

# rename Votes as actual
pred1 = pred1.rename(columns={"rank": 'actual'})

pred2 = predictions2[['predictions', 'actual', 'id']]

# order by id
pred2 = pred2.sort_values(by=['id'])

#pred2 = pred2.rename(columns={"rank": 'actual'})

# pred2['code'] = pred2['id'].str[:2]
# pred2['Year'] = pred2['id'].str[-4:]
# pred2 = pred2.drop(columns=['id'])

pred2.head(20)

In [ ]:
# combine the two dataframes
predictions_final = pd.concat([pred1, pred2])

predictions_final.head()

In [ ]:
predictions_final['predictions'].value_counts()

In [ ]:
violins(predictions_final)

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix
from scipy.stats import spearmanr

# confusion matrix for hurdle model

# evaluate predictions
accuracy = accuracy_score(predictions_final['predictions'], predictions_final['actual'])

cm = confusion_matrix(y_true=predictions_final['actual'], y_pred=predictions_final['predictions'])
plot = sns.heatmap(cm, annot=True, fmt='g', vmin=0, vmax=75)

print(len(cm[[0]][0]))

if(len(cm[[0]][0]) == 11) : 
# change the tick labels
    plot.set_xticks(np.arange(0, 11, 1)+0.5, np.arange(0, 9, 1).tolist() + [10, 12])
    plot.set_yticks(np.arange(0, 11, 1)+0.5, np.arange(0, 9, 1).tolist() + [10, 12])
else: 
    plot.set_xticks(np.arange(0, 10, 1)+0.5, np.arange(1, 9, 1).tolist() + [10, 12])
    plot.set_yticks(np.arange(0, 10, 1)+0.5, np.arange(1, 9, 1).tolist() + [10, 12])

# spearman correlation
corr, p = spearmanr(predictions_final['actual'], predictions_final['predictions'])

plot.text(x=6, y=-1.5, s= "XGBoost hurdle model", fontsize=12, ha="center")
plot.text(x=6, y=-0.5, s= "Accuracy=%.2f%%, r=%.2f"% (accuracy * 100.0, corr), fontsize=12, ha="center")
plot.set_ylabel('Actual Score')
plot.set_xlabel('Predicted Score')

plt.show()

# Predict the 2023 results

In [ ]:
# read in the csv
df_2023 = pd.read_csv('../../data/df_2023.csv')


df_2023['id'] = df_2023['from_code2'] + df_2023['year'].astype(str)
df_2023['received_vote'] = 0

df_2023 = df_2023[df_xgboost.columns]

In [ ]:
df_2023['from_code2'].value_counts().shape

In [ ]:
# Remove russia from the voting
df_2023 = df_2023.loc[df_2023['from_code2'] != 'RU', ]

In [ ]:
df_2023['from_code2']

df_2023['id'] = df_2023['from_code2'] + df_2023['year'].astype(str)

In [ ]:
df_hurdle = df_2023.copy()

# one hot encode the gender, from country and code
for j in ['gender', 'to_code2', 'from_code2']:
    j_text = "_voting" if j == 'from_code2' else ''
    for i in df_hurdle[j].unique():
        df_hurdle[i+j_text] = df_hurdle[j].apply(lambda x: 1 if x == i else 0)

    df_hurdle = df_hurdle[df_hurdle.columns.drop(j)]

#df2= df2[df2.columns.drop('From country')]
df_hurdle= df_hurdle[df_hurdle.columns.drop('year')]

df_hurdle.head()

In [ ]:
# print the test_data_all columns which are not in df_hurdle
print([col for col in df_hurdle.columns if col not in test_data_all.columns])
print([col for col in test_data_all.columns if col not in df_hurdle.columns])

# add the missing columns
for col in test_data_all.columns:
    if col not in df_hurdle.columns:
        df_hurdle[col] = 0

df_hurdle = df_hurdle[test_data_all.columns]

df_hurdle['rank'] = 1

df_hurdle.head()


In [ ]:
#model_ranked_all, test_data_all, train_data_all = xgboost_rank_model(df_2023, seed=7, test_size=0.33)

#sub_test = df_hurdle.loc[df_hurdle['id'] == i, ~df_hurdle.columns.isin(['rank', 'prediction_rel', 'predictions'])]

out_all = ranked_model_predictions(model_ranked_all, df_hurdle)

In [ ]:
out_all['predictions'].value_counts()

In [ ]:
p1 = out_all.copy()


cols = [col for col in p1.columns if '_voting' in col]
cols2 = [col for col in p1.columns if '_voting' not in col]


# wide format _voting columns to long format
p1 = pd.melt(p1, id_vars=cols2, 
                       value_vars= cols, var_name='from_code2')
p1 = p1.loc[p1['value'] == 1]


p1['from_code2'] = p1['from_code2'].str.replace('_voting', '').tolist()

p1 = p1.loc[p1['value'] == 1]
p1 = p1.drop('value', axis=1)



cols2 = [#'year', 
       'from_code2', #'to_code2', 
       #'rank',
       'Contains_English', 'id',
       'has_border', #'value', 
       'prediction_rel',
       'Contains_NonEnglish', 'Contains_Multiple_Languages',
       'Number_of_Languages', 'Contains_Own_Language', 'Contains_Voting_Language', 
       'prop_emigrants_v2p',  'prop_emigrants_p2v', 'rank',
       'comps_without_win', 'group', 'female', 'male', 
       'predictions', 'actual']

cols = [col for col in p1.columns if col not in cols2]

# # print p1.columns not in cols2
print([col for col in p1.columns if col not in cols2])

p1 = pd.melt(p1, id_vars=cols2, 
                       value_vars= cols, var_name='to_code2')
p1 = p1.loc[p1['value'] == 1]
# # drop value column
p1 = p1.drop('value', axis=1)

p1.head()

In [ ]:
p1['total_points'] = p1.groupby('to_code2')['predictions'].transform('sum')

p1['total_points'].value_counts()

In [ ]:
# results

res = p1[['total_points', 'to_code2']].drop_duplicates()


# match the country name to the country code
res = res.merge(df[['from_code2', 'from_country']], left_on='to_code2', right_on='from_code2', how='left').drop_duplicates()

res['position'] = res['total_points'].rank(ascending=False)

# sort by total_points
res = res.sort_values(by=['total_points'], ascending=False)

# rename from country to country
res = res.rename(columns={'from_country': 'country'})

# remove from_code2 column
res = res.drop('from_code2', axis=1)

# print without index
print(res.to_string(index=False))
